Text Mining - 6. Clasificacion textos

AFI - Máster en Data Science y Big Data

Juan de Dios Romero Palop

Abril 2022

Source: Andrew Task, Udacity


### 1. Carga de datos

In [1]:
## Críticas de películas
g = open('../reviews.txt','r') 
reviews = g.read().splitlines()
g.close()

## Sentimiento asociado
g = open('../labels.txt','r') # What we WANT to know!
labels = g.read().upper().splitlines()
g.close()

In [2]:
len(reviews)

25000

In [3]:
reviews[1]

'story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turned into an insane  violent mob by the crazy chantings of it  s singers . unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting . even those from the era should be turned off . the cryptic dialogue would make shakespeare seem easy to a third grader . on a technical level it  s better than you might think with some good cinematography by future great vilmos zsigmond . future stars sally kirkland and frederic forrest can be seen briefly .  '

In [4]:
len(labels)

25000

In [5]:
labels[1]

'NEGATIVE'

In [6]:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

In [7]:
print("labels.txt \t : \t reviews.txt\n")
pretty_print_review_and_label(2137)
pretty_print_review_and_label(12816)
pretty_print_review_and_label(6267)
pretty_print_review_and_label(21934)
pretty_print_review_and_label(5297)
pretty_print_review_and_label(4998)

labels.txt 	 : 	 reviews.txt

NEGATIVE	:	this movie is terrible but it has some good effects .  ...
POSITIVE	:	adrian pasdar is excellent is this film . he makes a fascinating woman .  ...
NEGATIVE	:	comment this movie is impossible . is terrible  very improbable  bad interpretat...
POSITIVE	:	excellent episode movie ala pulp fiction .  days   suicides . it doesnt get more...
NEGATIVE	:	if you haven  t seen this  it  s terrible . it is pure trash . i saw this about ...
POSITIVE	:	this schiffer guy is a real genius  the movie is of excellent quality and both e...


### 2. Análisis cuantitativo términos: ¿Qué términos aparecen en los comentarios positivos, cuales en los negativos y cuales aparecen en ambos?

In [8]:
from collections import Counter
import numpy as np

Vamos a utilizar la estructura Counter de python para ver cuantas veces aparece cada palabra en las críticas. Debajo tienes un ejemplo de como utilizar un contador.

https://docs.python.org/2/library/collections.html#collections.Counter

In [9]:
ex_count = Counter()
ex_count['palabra1'] = 3
ex_count['palabra2'] = 5
ex_count.most_common()

[('palabra2', 5), ('palabra1', 3)]

In [10]:
# Un contandor para cada tipo de review y uno total
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [11]:
# Crea un bucle que, para cada crítica, recorra sus palabras una a una e incremente en 1 el número de aparaciones.
# Aumenta el contador siempre en total_counts y en positive_counts O en negative_counts dependiendo de si es una crítica
# positiva o negativa
for i in range(len(reviews)):
  for word in reviews[i].split(' '):   
    if labels[i] == 'NEGATIVE':
      negative_counts[word] += 1
    else: 
      positive_counts[word] += 1
    total_counts[word] += 1


In [12]:
positive_counts.most_common(10)

[('', 550468),
 ('the', 173324),
 ('.', 159654),
 ('and', 89722),
 ('a', 83688),
 ('of', 76855),
 ('to', 66746),
 ('is', 57245),
 ('in', 50215),
 ('br', 49235)]

In [13]:
negative_counts.most_common(10)

[('', 561462),
 ('.', 167538),
 ('the', 163389),
 ('a', 79321),
 ('and', 74385),
 ('of', 69009),
 ('to', 68974),
 ('br', 52637),
 ('is', 50083),
 ('it', 48327)]

In [14]:
total_counts.most_common(10)

[('', 1111930),
 ('the', 336713),
 ('.', 327192),
 ('and', 164107),
 ('a', 163009),
 ('of', 145864),
 ('to', 135720),
 ('is', 107328),
 ('br', 101872),
 ('it', 96352)]

El resultado del conteo muestra que las stopwords están presentes tanto en críticas positivas como en críticas negativas y pueden añadir ruido a la hora crear un modelo de clasificación. ¿Cómo podemos sacar aquellas palabras que son un indicador claro de que se trata de una crítica positiva o negativa? 

### 3. Análisis cuantitativo términos: Ratios

Vamos a calcular los ratios de aparición de los términos de la siguiente manera: ratio = positive_counts/float(negative_counts + 1).

**Nota**: vamos a trabajar unicamente con aquellos términos que **en total** aparecen 101 o más veces.

*   ¿Por qué ese +1 en el denominador?
*   A bote pronto, ¿cómo interpretaríamos los resultados? ¿En qué rango se van a mover los ratios?



In [15]:
pos_neg_ratios = Counter()

# Calcula el ratio para los término más comunes
for term,cnt in list(total_counts.most_common()):
  ### Inserta tu codigo aqui
  if cnt > 100:
    pos_neg_ratios[term] = positive_counts[term]/float(negative_counts[term] + 1)
  else:
    break

In [16]:
pos_neg_ratios.most_common(15)

[('edie', 109.0),
 ('paulie', 59.0),
 ('felix', 23.4),
 ('polanski', 16.833333333333332),
 ('matthau', 16.555555555555557),
 ('victoria', 14.6),
 ('mildred', 13.5),
 ('gandhi', 12.666666666666666),
 ('flawless', 11.6),
 ('superbly', 9.583333333333334),
 ('perfection', 8.666666666666666),
 ('astaire', 8.5),
 ('captures', 7.68),
 ('voight', 7.615384615384615),
 ('wonderfully', 7.552631578947368)]

In [17]:
## Algunos ejemplos que nos ayudan a interpretar los resultados
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 1.0607993145235326
Pos-to-neg ratio for 'amazing' = 4.022813688212928
Pos-to-neg ratio for 'terrible' = 0.17744252873563218


¿Qué problema tiene esta definición de ratio? ¿Cómo lo solucionamos?

### 4. Análisis cuantitativo términos: Logaritmos

Al aplicar logaritmos a los valores calculados en el apartado anterior hacemos que los valores por debajo de 1 pasen a ser negativos (y con valor absoluto más alto cuanto más cercanos a 0 sean) y además conseguimos que dos términos con frecuencias relativas parecidas pero en críticas de signo distinto tomen valores con valor absoluto parecido y signo contrario.

In [18]:
# Calcula el logaritmo de los ratios para todos los términos
## Inserta tu código aqui
for term, cnt in list(pos_neg_ratios.most_common()):
    pos_neg_ratios[term] = np.log(cnt) 

In [19]:
pos_neg_ratios.most_common(15)

[('edie', 4.6913478822291435),
 ('paulie', 4.07753744390572),
 ('felix', 3.152736022363656),
 ('polanski', 2.8233610476132043),
 ('matthau', 2.80672172860924),
 ('victoria', 2.681021528714291),
 ('mildred', 2.6026896854443837),
 ('gandhi', 2.538973871058276),
 ('flawless', 2.451005098112319),
 ('superbly', 2.26002547857525),
 ('perfection', 2.159484249353372),
 ('astaire', 2.1400661634962708),
 ('captures', 2.038619547159581),
 ('voight', 2.030170492673053),
 ('wonderfully', 2.0218960560332353)]

In [20]:
## Algunos ejemplos que nos ayudan a interpretar los resultados
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

Pos-to-neg ratio for 'the' = 0.05902269426102881
Pos-to-neg ratio for 'amazing' = 1.3919815802404802
Pos-to-neg ratio for 'terrible' = -1.7291085042663878


In [21]:
pos_neg_ratios.most_common()[:-31:-1]

[('boll', -4.969813299576001),
 ('uwe', -4.624972813284271),
 ('seagal', -3.644143560272545),
 ('unwatchable', -3.258096538021482),
 ('stinker', -3.2088254890146994),
 ('mst', -2.9502698994772336),
 ('incoherent', -2.9368917735310576),
 ('unfunny', -2.6922395950755678),
 ('waste', -2.6193845640165536),
 ('blah', -2.5704288232261625),
 ('horrid', -2.4849066497880004),
 ('pointless', -2.4553061800117097),
 ('atrocious', -2.4259083090260445),
 ('redeeming', -2.3682390632154826),
 ('prom', -2.3608540011180215),
 ('drivel', -2.3470368555648795),
 ('lousy', -2.307572634505085),
 ('worst', -2.286987896180378),
 ('laughable', -2.264363880173848),
 ('awful', -2.227194247027435),
 ('poorly', -2.2207550747464135),
 ('wasting', -2.204604684633842),
 ('remotely', -2.1972245773362196),
 ('existent', -2.0794415416798357),
 ('boredom', -1.995100393246085),
 ('miserably', -1.9924301646902063),
 ('sucks', -1.987068221548821),
 ('uninspired', -1.9832976811269336),
 ('lame', -1.981767458946166),
 ('insult

### 5. Modelo de clasificación basado en bag of words

Vamos a aplicar la técnica de bag of words paso a paso a cada una de las críticas con el objetivo de convertirlas en vectores numéricos que sirvan de features de nuestro modelo.

#### Primer paso: construir el conjunto de palabras de nuestro vocabulario.

In [22]:
vocab = set(total_counts.keys())

In [23]:
vocab_size = len(vocab)
vocab_size

74074

#### Segundo paso: construímos un vector del tamaño de nuestro vocabulario. Para ganar tiempo lo creamos como un vector entero de 0s usando la función de numpy zeros()

In [24]:
import numpy as np
zeros = np.zeros(vocab_size)

In [25]:
zeros

array([0., 0., 0., ..., 0., 0., 0.])

#### Tercer paso: asignar a cada palabra un índice del vector y crear una tabla maestra que guarde esta relación y nos permita crear fácilmente nuestros vectores.

In [26]:
# Creamos un diccionario que tiene como key la palabra y como valor el índice asociado
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i
    
word2index

{'': 0,
 'doubly': 1,
 'converge': 2,
 'tunnel': 3,
 'roemheld': 4,
 'warlike': 5,
 'hool': 6,
 'forefathers': 7,
 'proposing': 8,
 'defunct': 9,
 'perverse': 10,
 'wazoo': 11,
 'menagerie': 12,
 'grody': 13,
 'delaware': 14,
 'disassociative': 15,
 'restoration': 16,
 'konishita': 17,
 'spotlights': 18,
 'enyclopedia': 19,
 'hill': 20,
 'recalled': 21,
 'cloth': 22,
 'hares': 23,
 'conclusiondirector': 24,
 'grounding': 25,
 'pheasant': 26,
 'newsreel': 27,
 'galton': 28,
 'oodishon': 29,
 'kairo': 30,
 'kho': 31,
 'slave': 32,
 'homeless': 33,
 'wowsers': 34,
 'hoss': 35,
 'jutta': 36,
 'creaks': 37,
 'neagle': 38,
 'techno': 39,
 'sandbox': 40,
 'absolutey': 41,
 'complaint': 42,
 'cherri': 43,
 'oiks': 44,
 'tugged': 45,
 'digby': 46,
 'morvern': 47,
 'bingham': 48,
 'sargeants': 49,
 'larky': 50,
 'therethat': 51,
 'healthy': 52,
 'survives': 53,
 'shabby': 54,
 'harpy': 55,
 'steady': 56,
 'horor': 57,
 'underlies': 58,
 'locating': 59,
 'katzelmacher': 60,
 'lovableness': 61,
 '

#### Cuarto paso: crear la función que dada una crítica devuelve un vector contando la frecuencia de las palabras utilizadas.

In [133]:
## Rellena la función que para cada crítica devuelve el vector asociado aplicando bag of words
def bag_of_words(review):
  v = np.zeros(vocab_size)
  ## Inserta tu código aquí
  for word in review.split(' '):
    v[word2index[word]] += 1
  return v

In [28]:
reviews[0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   '

In [145]:
word2index['']

0

In [144]:
word2index["bromwell"]

13982

In [139]:
bag_of_words(reviews[0])

array([18.,  0.,  0., ...,  0.,  0.,  0.])

In [147]:
bag_of_words(reviews[0])[13982]

4.0

In [33]:
bag_of_words(reviews[0]).shape

(74074,)

#### Quinto paso: el target de nuestro modelo serán 1s y 0s. Vamos a crear una función que convierta los cadenas POSITIVE y NEGATIVE a 1 y 0. 

In [34]:
## Rellena la función para que dada la etiqueta en forma de cadena devuelve el entero asociado
def target_numerico(label):
    return int(label == 'POSITIVE')

In [35]:
target_numerico(labels[0])

1

#### Sexto paso: dividimos los datos que tenemos en train y test (esta vez lo hacemos a ojo).

In [36]:
training_size = 5000
test_size = 10000
training_rev = reviews[:training_size]
training_lab = labels[:training_size]
test_rev = reviews[-test_size:]
test_lab = labels[-test_size:]
print(len(training_lab))
print(len(test_lab))

5000
10000


#### Séptimo paso: calculamos las matrices de entrenamiento y de test.

In [37]:
X = np.empty((len(training_rev), vocab_size))
print(X.shape)
### Rellena X aquí
for i in range(len(training_rev)):
    X[i] = bag_of_words(training_rev[i])

(5000, 74074)


In [149]:
X_test = np.empty((len(test_rev), vocab_size))
print(X_test.shape)
for i in range(len(test_rev)):
    X_test[i] = bag_of_words(test_rev[i])

(10000, 74074)


In [39]:
y = np.empty((len(training_lab),))
print(y.shape)
# Rellena y aquí

for i in range(len(training_rev)):
    y[i] = target_numerico(training_lab[i])

(5000,)


In [40]:
y_test = np.empty((len(test_lab),))
print(y_test.shape)
# Rellena y_test aquí

for i in range(len(test_lab)):
    y_test[i] = target_numerico(test_lab[i])

(10000,)


#### Octavo paso: entrenamos el modelo

In [41]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [42]:
model = linear_model.LogisticRegression()
model.fit(X,y)

C:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

#### Noveno paso: aplicamos el predict sobre el conjunto de test y vemos qué tal funciona.

In [43]:
predictions = model.predict(X_test)

In [44]:
model.score(X,y)

0.9988

In [45]:
model.score(X_test,y_test)

0.8402

In [46]:
print(confusion_matrix(y_test, predictions))

[[4310  690]
 [ 908 4092]]


In [47]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.83      0.86      0.84      5000
         1.0       0.86      0.82      0.84      5000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



#### Decimo paso: preparamos el código para probar el modelo con cadenas nuevas.

In [104]:
# Rellena la función para, dada una crítica, aplicar el modelo que hemos entrenado
# e imprimir POSITIVE/NEGATIVE
def sentiment_analysis(review):
  ## Inserta tu código aqui
  vector_input = np.empty((1, vocab_size))
  vector_input[0] = bag_of_words(review)
  pred = model.predict(vector_input)
  if pred == 1:
    return 'POSITIVE'
  else:
    return 'NEGATIVE'

In [105]:
sentiment_analysis('movie bad')

'NEGATIVE'

In [126]:
sentiment_analysis('not horrible')

Words not found:  0


'NEGATIVE'

In [123]:
sentiment_analysis('España')

Words not founds:  1


'POSITIVE'

¿Qué hacemos con el error que se obtiene al meter una palabra que no está en el vocabulario? Solucionar este error es parte de la práctica de la asignatura.

In [130]:
sentiment_analysis('Cool')

Words not found:  0


'POSITIVE'

Pero si Cool si es una palabra inglesa. ¿Qué ocurre? ¿Cómo lo solucionamos? Esto también es parte de la práctica de la asignatura. 

### Práctica

Modificaremos la función bag_of_words para que, primero, convierta en minúsculas la review que introducimos y, segundo, añada un condicional que compruebe si existen las palabras de la review en nuestro diccionario. Además, crearemos un contador para imprimir por pantalla cuantas palabras no se encontraban en el diccionario.

In [ ]:
## Rellena la función que para cada crítica devuelve el vector asociado aplicando bag of words
def bag_of_words(review):
  review = review.lower()
  v = np.zeros(vocab_size)
  words_not_founds = 0
  ## Inserta tu código aquí
  for word in review.split(' '):
    if word not in word2index:
        words_not_founds += 1
    else:
        v[word2index[word]] += 1
  print(f'Words not found:  {words_not_founds}')
  return v

In [131]:
sentiment_analysis('España')

Words not found:  1


'POSITIVE'

In [132]:
sentiment_analysis('Cool')

Words not found:  0


'POSITIVE'